# RSA：惊艳

## 1. Simple RSA
给定 ($N = p * q, p, q, e$)，进行消息的加解密：

In [2]:
from Crypto.Util.number import getPrime, long_to_bytes, bytes_to_long
import gmpy2
p = getPrime(2048)
q = getPrime(2048)
N = p * q
e = 65535

msg = b"z2o-k7e_guys"
plain = bytes_to_long(msg)

c = pow(plain, e ,N)

# compute inverse modulos N of e
phiN = (p - 1) * (q - 1)
d = pow(e, -1, phiN)

print(long_to_bytes(pow(c, d, N)))

b'z2o-k7e_guys'


## 2. Attack vectors
### I. 已知e,p,q,c

直接求 $e$ 在模 $\varphi(N)$ 的逆元，解密即可。

这个攻击成立的前提是：大整数分解难题被“攻破”。

目前有三个工具可以尝试分解 N ：

- [Yafu](https://github.com/bbuhrow/yafu): 适用于 $p$ 和 $q$ 接近或者相差太大的情况。
- [dbfactor](http://www.factordb.com/): 更加通用的工具，但是位数有限。
- [sagemath](https://doc.sagemath.org/html/en/reference/interfaces/sage/interfaces/ecm.html#sage.interfaces.ecm.ECM.factor): sage自带的分解工具。

因此，当选择安全的 $p$ 和 $q$ 参数后，这种攻击可以被规避。

### II. 已知e,n,c,dp（或dq）其中n很大，无法分解

以dp为例： $dp \equiv d\mod(p-1)$ 。

就有： $dp*e \equiv d*e\mod(p-1)$.可以写作：$dp*e=d*e+k_1*(p-1)$ 。

而 $d*e \equiv 1\mod phi(n)$ .可以写作： $d*e=1+k_2*phi(n)$ ，代入上式有：

$$
dp * e = 1 + k_2*phi(n)+k_1*(p-1)
$$

不妨写作： $dp*e+k_1*(p-1) =1+k_2*(p-1)*(q-1)$ ，

移项有： $dp*e=1+(p-1)*[k_2*(q-1)-k_1]$ ，

为了方便观察，设 $x=k_2*(q-1)-k_1$ ，则上式有： $dp*e=1+(p-1)*x$ 。

由于 $dp \lt p-1$ ，则 $x \lt e $，那么我们就可以遍历 $0\lt x\lt e$ ，那么就可以求得 $p-1 = (dp * e - 1) / x$ 。


In [4]:
# e, n, dp, c已知
from gmpy2 import *
def getD(e,n,dp):
    for i in range(1,e):
        if (dp*e-1)%i == 0 and n % ((dp*e-1)%i+1) == 0:
            p = (dp*e-1)%i+1
            q = n % p
            return invert(e, (p-1)*(q-1))


### III. 已知n,c1,c2,e1,e2(公共模数攻击)

两个及以上的公钥 $(n,e)$ 来加密同一条信息 $m$:

$$
c1 = pow(m, e1, n) \\
c2 = pow(m, e2, n)
$$

且 $(e1, e2)$ 互素 $gcd(e1, e2) = 1$ ，根据扩展欧几里德算法对于不完全为 $0$ 的整数 $a$， $b$ ， $gcd(a, b)$ 表示 $a$， $b$ 的最大公约数。那么一定存在整数 $x$， $y$ 使得 $gcd(a, b)=ax+by$ 。

由 $c1 = m^{e1} \pmod{n}, c2 = m^{e2}\pmod{n}$ ，得: $(c1^{s1}*c2^{s2})\%n = ((m^{e1}\%n)^{s1}(m^{e2}\%n)^{s2})\%n$ .

```python
(c1^s1*c2^s2)%n = ((m^e1%n)^s1*(m^e2%n)^s2)%n

=> (c1^s1*c2^s2)%n = ((m^e1%n)^s1%n*(m^e2%n)^s2%n)%n #(a * b) % p = (a % p * b % p) % p

=> (c1^s1*c2^s2)%n = ((m^e1)^s1%n*(m^e2)^s2%n)%n #((a % p) ^ b) % p =a ^ b % p

=> (c1^s1*c2^s2)%n = ((m^e1)^s1*(m^e2)^s2)%n # (a % p * b % p) % p=(a * b) % p

=>(c1^s1*c2^s2)%n = ((m^(e1*s1)*(m^(e2*s2))%n #。幂的乘方，底数不变，指数相乘。

(c1^s1*c2^s2)%n = (m^(e1*s1+e2*s2))%n  # 同底数幂相乘，底数不变，指数相加。
```

由于 $e1*s1+e2*s2 = 1$ 得： $({c1}^{s1}*{c2}^{s2})\%n = (m^1)\%n$ 。即可恢复明文。

题目：

In [7]:
import libnum
import gmpy2
#生成素数
p=libnum.generate_prime(1024)
q=libnum.generate_prime(1024)
e1=2333
e2=23333
m="flag{6ed4c74e022cb18c8039e96de93aa9ce}"
m=libnum.s2n(m)
n=p*q
c1=pow(m,e1,n)
c2=pow(m,e2,n)
print("n1=",n)
print("e1=",e1)
print("c1=",c1)
print("n2=",n)
print("e2=",e2)
print("c2=",c2)

n1= 18549813963079476875632853308673997827094591535253238910783812972563030969419070400764607127322340033061563145714644991409806347178948591809519796907541756737435993764103805854674126992210575421778371746255261069135144227868472820869921634341685400825662266412243974956616119460116514546445133254261394331598810174347776700512787784835718088336092367518182228844137764431234750024935273753163861892294142136005492930143225642638795214845780254738529534896031874075431666512202926615329356680999189662749059602199401779874975328208817874617158872430337386259447940111178501486937426608628679495767208940740280777369493
e1= 2333
c1= 94043319846831589800554454793151244414984815123150302791080390670949671960320266081121543526303718976123674878603983921088298653989152686268663947790935719862283902631768425935074900674989825022726589125182266558430451322843704819847876709962355972230119144832212200304444996192879276846080000152809244971272170539668649148987463757609625845151027463690203239660884

In [8]:
import gmpy2
import libnum

n= 25333966058003377512707481338795714713737652659944601834182685873529702913988641983855700459996104700470621411559153944988952210084014634675583566338568882440708528997808798650962116756969822211586531522430245040013570571043385238601846104615050089457836721437790715225367971106085839523500735480715155424498941150468093083816830215632716244390679417218873179734276657411907216486790815037105108306055794473002315541787461904728375164737225486501009857299717749346279371251245318729951905832578739696926931502225899707226264570557623527701806829827566904224572897378639191756878049342203546309520458672464170859577433
e1= 2333
c1= 11355981897781478907853356911752561659125575027336719997290835661089901154031171698660586203170528368228850895159361637188990782030255983633790580700032092629587631108597144196551438410868034739981960656110887650747325311613900008001234835897835613759692146419080113176963747835592656185435741504176116672174539018089139547795447109458469225330809064539216773123671814859510069089532677704482026169178543062578686794346026774853085101278125763460212801929360456888869350105294595904940799522522144740464043605342348269086324747729288399275079874271519208155039364092577755518532799345651172433227745483422620900776136
e2= 23333
c2= 1326499538902841116411674554069945581390130048432351353260154261863309471312810811160302458644816390944833633923435634961251092531893503039914793217247984595331920909367627316087404934402312358642315675486438968585084964845763881078835787872160374025547400141298650794551970291119975344578667689961134814676553190178139842507675899262024572370313939107080072875068218336255452161407859907308656094331557912187788276334833723893856310434523337557011032081467262457427167978528280339494077785813461280853735266463152709443731638714219391773144349752633555310299318290576258086971373777118482642702020599928071855133041

#共模攻击
#共模攻击函数
def rsa_gong_N_def(e1,e2,c1,c2,n):
    e1, e2, c1, c2, n=int(e1),int(e2),int(c1),int(c2),int(n)
    s = gmpy2.gcdext(e1, e2)
    s1 = s[1]
    s2 = s[2]
    if s1 < 0:
        s1 = - s1
        c1 = gmpy2.invert(c1, n)
    elif s2 < 0:
        s2 = - s2
        c2 = gmpy2.invert(c2, n)
    m = (pow(c1,s1,n) * pow(c2 ,s2 ,n)) % n
    return int(m)
m = rsa_gong_N_def(e1,e2,c1,c2,n)
print(m)
print(libnum.n2s(int(m)))

13040004482820198698847089094244152075790400869393100003271655421286266920976016007292478845
b'flag{6ed4c74e022cb18c8039e96de93aa9ce}'


### IV. 已知n（非常大）,e,d，求p、q


In [18]:
import random
from hashlib import md5
 
def gcd(a, b):
   if a < b:
     a, b = b, a
   while b != 0:
     temp = a % b
     a = b
     b = temp
   return a
 
def getpq(n,e,d):
	p = 1
	q = 1
	while p==1 and q==1:
		k = d * e - 1
		g = random.randint ( 0 , n )
		while p==1 and q==1 and k % 2 == 0:
			k //= 2
			y = pow(g,k,n)
			if y!=1 and gcd(y-1,n)>1:
				p = gcd(y-1,n)
				q = n/p
	return p,q
 
def main():
	n = 16352578963372306131642407541567045533766691177138375676491913897592458965544068296813122740126583082006556217616296009516413202833698268845634497478988128850373221853516973259086845725813424850548682503827191121548693288763243619033224322698075987667531863213468223654181658012754897588147027437229269098246969811226129883327598021859724836993626315476699384610680857047403431430525708390695622848315322636785398223207468754197643541958599210127261345770914514670199047435085714403641469016212958361993969304545214061560160267760786482163373784437641808292654489343487613446165542988382687729593384887516272690654309
	e = 65537
	d = 9459928379973667430138068528059438139092368625339079253289560577985304435062213121398231875832264894458314629575455553485752685643743266654630829957442008775259776311585654014858165341757547284112061885158006881475740553532826576260839430343960738520822367975528644329172668877696208741007648370045520535298040161675407779239300466681615493892692265542290255408673533853011662134953869432632554008235340864803377610352438146264524770710345273439724107080190182918285547426166561803716644089414078389475072103315432638197578186106576626728869020366214077455194554930725576023274922741115941214789600089166754476449453
	p, q = getpq(n,e,d)
	print(p)
	print(q)
	print("Flag: flag{%s}" %md5(str(p + q).encode()).hexdigest())

if __name__ == '__main__':
	main()

121681461613856685891389697655082707982324934394003375745821514797619569583750841725694490585739982440237839675155146102668334623474524757328414864963814738071946391260695792366762521733527504377788503669628114869921159658618293030663730923781347146576731525405173348568491361155907470541865888995846800200299
1.3438841666173846e+308
Flag: flag{ee7b630995e7a36b6420696989441e2d}


### V. 广播攻击Broadcast attcak

脚本：

```python
from math import gcd
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.number import *

c,n = [],[]
e = 0x10001


def crack(ind1,ind2):
    n1,n2 = n[ind1],n[ind2]
    p = gcd(n1,n2)
    q = n1 // p
    phin = p*q-p-q+1
    d = pow(e,-1,phin)
    k = RSA.construct((n1,e,d,p,q))
    kk = PKCS1_OAEP.new(k)
    flag = kk.decrypt(c[ind1])
    print(flag)
    return flag
    
for i in range(10):
    for j in range(i+1,10):
        if gcd(n[i],n[j]) != 1:
            crack(i,j)
```

### VI. p-1光滑&&p+1光滑

- $p - 1$ smooth

    ```python
    from gmpy2 import *
    a = 2
    n = 2
    while True:
        a = powmod(a, n, N)
        res = gcd(a-1, N)
        if res != 1 and res != N:
            q = N // res
            d = invert(e, (res-1)*(q-1))
            m = powmod(c, d, N)
            print(m)
            break
        n += 1
    ```

- $p + 1$ smooth

    ```python
    def mlucas(v, a, n):
        """ Helper function for williams_pp1().  Multiplies along a Lucas sequence modulo n. """
        v1, v2 = v, (v**2 - 2) % n
        for bit in bin(a)[3:]: v1, v2 = ((v1**2 - 2) % n, (v1*v2 - v) % n) if bit == "0" else ((v1*v2 - v) % n, (v2**2 - 2) % n)
        return v1

    for v in count(1):
        for p in primegen():
            e = ilog(isqrt(n), p)
            if e == 0: break
            for _ in xrange(e): v = mlucas(v, p, n)
            g = gcd(v-2, n)
            if 1 < g < n: return g # g|n
            if g == n: break
    ```

### VII. 低加密指数攻击（e太小）

- 直接开方

    ```python
    import gmpy2
    m = gmpy2.iroot(c,e)[0]
    ```

- rabin算法(e = 2)

    ```python
    def rabin_decrypt(c, p, q, e=2):  
        n = p * q  
        mp = pow(c, (p + 1) / 4, p)  
        mq = pow(c, (q + 1) / 4, q)  
        yp = gmpy2.invert(p, q)  
        yq = gmpy2.invert(q, p)  
        r = (yp * p * mq + yq * q * mp) % n  
        rr = n - r  
        s = (yp * p * mq - yq * q * mp) % n  
        ss = n - s  
        return (r, rr, s, ss) 
    ```

### VIII. 小明文攻击

- e = 3，m 远小于 N，则直接开三次方

- 模不互素

    给定两个模数 $N_1, N_2$ 有公因子，那么可以直接 $gcd(N_1,N_2)$ 求得公因子。

- e 和 phi 不互素

    e 和 $(p-1)$ 或 $(q-1)$ 不互素，那么就无法求 $d=invert(e,phi)$ 。

    设$x=gcd(e,phi)$，有两种情况：

    - e和phi的公因子$x$比较小（$m^x<n$）

        已知 $c=m^e\mod n$，可以写作$c=m^{x*(e//x)} \mod n$，其中//表示整除

        把$m^x$看作新的$m_1$，则$c=m_1^{e//x}\mod n$，此时$e//x$与phi是互质的，将两者模逆之后求出来的明文实际上是$m^x$，所以需要开x次方才能得到真正的$m$

    - e和phi的公因子$x$比较大无法开方（$m^x>n$）

        尝试在有限域上开根：

        - rabin算法（在e=2时的有限域开根）

            同上

        - AMM算法（当phi//e时进行的有限域开根）

            https://tttang.com/archive/1504/

### IX. coppersmith攻击

1. 已知明文高位：Stereotyped messages

In [5]:
from Crypto.Util.number import *
n = 0x2519834a6cc3bf25d078caefc5358e41c726a7a56270e425e21515d1b195b248b82f4189a0b621694586bb254e27010ee4376a849bb373e5e3f2eb622e3e7804d18ddb897463f3516b431e7fc65ec41c42edf736d5940c3139d1e374aed1fc3b70737125e1f540b541a9c671f4bf0ded798d727211116eb8b86cdd6a29aefcc7
e = 3
m = randrange(n)
c = pow(m, e, n)
beta = 1
epsilon = beta^2/7
nbits = n.nbits()
kbits = floor(nbits*(beta^2/e-epsilon))
 
mbar = 0xb11ffc4ce423c77035280f1c575696327901daac8a83c057c453973ee5f4e508455648886441c0f3393fe4c922ef1c3a6249c12d21a000000000000000000
c = 0x1f6f6a8e61f7b5ad8bef738f4376a96724192d8da1e3689dec7ce5d1df615e0910803317f9bafb6671ffe722e0292ce76cca399f2af1952dd31a61b37019da9cf27f82c3ecd4befc03c557efe1a5a29f9bb73c0239f62ed951955718ac0eaa3f60a4c415ef064ea33bbd61abe127c6fc808c0edb034c52c45bd20a219317fb75
print ("upper %d bits (of %d bits) is given" % (nbits-kbits, nbits))

PR.<x> = PolynomialRing(Zmod(n))
f = (mbar + x)^e - c
x0 = f.small_roots(X=2^kbits, beta=1)[0]  # find root < 2^kbits with factor = n1
print (mbar + x0)

upper 828 bits (of 1022 bits) is given
2264840338413586905738035979327923800192834021938622714236017601553797129296483661774952688217311434324726663390730663270760957384056176091290984684529


2. 已知p的高位：Factoring with High Bits Known

In [7]:
n = 0x5894f869d1aecee379e2cb60ff7314d18dbd383e0c9f32e7f7b4dc8bd47535d4f3512ce6a23b0251049346fede745d116ba8d27bcc4d7c18cfbd86c7d065841788fcd600d5b3ac5f6bb1e111f265994e550369ddd86e20f615606bf21169636d153b6dfee4472b5a3cb111d0779d02d9861cc724d389eb2c07a71a7b3941da7dL
p_fake = 0x5d33504b4e3bd2ffb628b5c447c4a7152a9f37dc4bcc8f376f64000fa96eb97c0af445e3b2c03926a4aa4542918c601000000000000000000000000000000000

pbits = p_fake.nbits()
kbits = 128  #p失去的低位
pbar = p_fake & (2^pbits-2^kbits)
print ("upper %d bits (of %d bits) is given" % (pbits-kbits, pbits))
 
PR.<x> = PolynomialRing(Zmod(n))
f = x + pbar
x0 = f.small_roots(X=2^kbits, beta=0.4)[0]  # find root < 2^kbits with factor >= n^0.3
print (x0 + pbar)

from sage.all import *
h = 9605964225476901441398365225327926616880072280289780777971846998748464126891804587377933727304510424852546683782576240573278202121547956666293242671661056
h = (h>>128)
e = 65537
c = 2226099021169425534206121605501718994593261953280046899345810118356590881389142531649792348146129153474985003929407172972982275439970723778495455838452638879586163957468972518078320159354264971816842073874550773309020013613432004074760802192607651584906352686468143648939740004838208640531785439362344039075       
n = 96928253979490973984593903132811649229014718994486532280648145898877952846656019305217095845257550421730063527538581223570539203247068060192535543753763017716750817560470547219370972835770943358384150269303529653434434525449357699107332781898776312692702549420939758722366794431784782973884379040574148608179      
pbits = 512
kbits = pbits - h.nbits()
print(h.nbits())
h = h << kbits
PR.<x> = PolynomialRing(Zmod(n))
f = x + h
roots = f.small_roots(X=2^kbits, beta=0.4)
if roots:        
    p = h+int(roots[0])
    q = n//p
    assert(p*q==n)
    print("n= "+str(n))
    print("p= "+str(p))
    print("q= "+str(q))

upper 383 bits (of 511 bits) is given
4881303325607587234201385254050033835312172878514703272072771615943902178092736544523304388539705532310101139649392707401107616606921391984907633619668131
384
n= 96928253979490973984593903132811649229014718994486532280648145898877952846656019305217095845257550421730063527538581223570539203247068060192535543753763017716750817560470547219370972835770943358384150269303529653434434525449357699107332781898776312692702549420939758722366794431784782973884379040574148608179
p= 9605964225476901441398365225327926616880072280289780777971846998748464126891804587377933727304510424852546683782576502313782852920367078761087206532330667
q= 10090424209827705883904237600994309311141041458942536296570950440834978877897662247343765605047612897055764165403472353320267482450970721050565927821120537


3. 多因子

In [9]:
# sagemath

e = 65537
n = 36618139579386063246087882054063631367923586826293230665209915187491823328978276724908066032487515386697740611819366867179565337532194305783987450587518624526250530134446397
c = 3053043969587277731075013823380664207370991627277672374256662715889363487017560381573682876563907215099359894935326265406537547932246927604121814198201993671878573628633125

print(euler_phi(n))
# phi = 34759796809852833530701890199359568656326088873172579834205079714452124445487502116540486612345625847415073120247966210442818096013930370529609373694603238047744000000000000
phi = euler_phi(n)

from Crypto.Util.number import *

d = inverse(e,phi)
m = pow(c,d,n)
print(long_to_bytes(m))
print(long_to_bytes(m).decode())
# NSSCTF{3eea2df4-45e5-11ed-b29a-28d0eab06969}

34759796809852833530701890199359568656326088873172579834205079714452124445487502116540486612345625847415073120247966210442818096013930370529609373694603238047744000000000000
b'NSSCTF{3eea2df4-45e5-11ed-b29a-28d0eab06969}'
NSSCTF{3eea2df4-45e5-11ed-b29a-28d0eab06969}


4. 已知d的低位：Partial Key Exposure Attack

In [10]:
def partial_p(p0, kbits, n):
    PR.<x> = PolynomialRing(Zmod(n))
    nbits = n.nbits()
    f = 2^kbits*x + p0
    f = f.monic()
    roots = f.small_roots(X=2^(nbits//2-kbits), beta=0.3)  # find root < 2^(nbits//2-kbits) with factor >= n^0.3
    if roots:
        x0 = roots[0]
        p = gcd(2^kbits*x0 + p0, n)
        return ZZ(p)
def find_p(d0, kbits, e, n):
    X = var('X')
    for k in range(1, e+1):
        results = solve_mod([e*d0*X - k*X*(n-X+1) + k*n == X], 2^kbits)
        for x in results:
            p0 = ZZ(x[0])
            p = partial_p(p0, kbits, n)
            if p:
                return p
if __name__ == '__main__':
    n = 0xd463feb999c9292e25acd7f98d49a13413df2c4e74820136e739281bb394a73f2d1e6b53066932f50a73310360e5a5c622507d8662dadaef860b3266222129fd645eb74a0207af9bd79a9794f4bd21f32841ce9e1700b0b049cfadb760993fcfc7c65eca63904aa197df306cad8720b1b228484629cf967d808c13f6caef94a9
    e = 3
    d = 0x603d033f2ef6c759aec839f132a45215fc8a635b757f3951a731fe60bc6729b3bcf819b57abfcaba3a93e9edef766c0d499cad3f7adb306bcf1645cfb63400e3
    beta = 0.5
    epsilon = beta^2/7
    nbits = n.nbits()
    print ("nbits:%d:"%(nbits))
    #kbits = floor(nbits*(beta^2+epsilon))
    kbits = nbits - d.nbits()-1
    print ("kbits:%d"%(kbits))
    d0 = d & (2^kbits-1)
    print ("lower %d bits (of %d bits) is given" % (kbits, nbits))
    p = find_p(d0, kbits, e, n)
    print ("found p: %d" % p)
    q = n//p
    print (d)
print (inverse_mod(e, (p-1)*(q-1)))

nbits:1024:
kbits:512
lower 512 bits (of 1024 bits) is given
found p: 11225675567380871009570101516613347989321681194686379697181792843475612230559123178043068043743341164294959005139360852241393871718562719024517338855867479
5040410370166817540949312087060415045446840370221425634233624136691492241375585921058583488309075121656628872540400242449616270663369223294154675169001699
99430503515013903040291384133470968970466103991571343492764814780967090203742450771166104729697611017493494950675760654484422260868428264884630250537959791201255204527110189941039822089612031446149334262602063427352960800355020085981806048749599222109448518162553849621230522535730119500009520776075508187363


5. Related Message Attack

In [ ]:
import gmpy2
import functools

def getM2(a,b,c1,c2,n):
    a3 = pow(a,3,n)
    b3 = pow(b,3,n)
    first = c1-a3*c2+2*b3
    first = first % n
    second = 3*b*(a3*c2-b3)
    second = second % n
    third = second*gmpy2.invert(first,n)
    third = third % n
    fourth = (third+b)*gmpy2.invert(a,n)
    return fourth % n
a=1
b=-1
padding2=b
n=0xf9526aad4d41c9b28f8bae279c7ef6b07d729d1f56e530219851f656ad521218815bdccb15167a25633a2f76969fccd3fe1ef379ded08d1a9c3307f680e952956d2b3d04cc50040efb30e40bf2562aae4b05b8ec0d5e0e0ea5fdc1b00b80dee9b6de1d77d41d8d040d3465c89133d9af23b1d43f57e70606e3433d35a47e2ed
c1=0x798841c574b7c88ce1430d4b02bac01fc9368c71a7966176b22f9dc2e0c2f6d4d5b8a9e10dbcaa4584e667ef1afd213b78c2bdc16ba5ab909c2de2fe5a7a5fa36a390bdccf794451cd9db8489ed7870efa4a4d7d1cacacfec92e81f6bb955a4ef5d71d80631c0726d22ec3d5b115de7ff42f22e67854b59ed816e06485ab523
c2=0xe92c4c99052fa3c4bb5e54477b0afe8e18da37255269f070ffa6824492a87153e428fa4ed839b7f3249966259a0c88641185594fc2fa4881cf32b7af5b18baa6f5200453ee80e38c74dbeb90f32118e4f33e636a808e44f27e09286d109ee8f41765ad64c7afea9775974d78a80e0977a37689c7f15a23a83a87b1f5bdbcdec
m = getM2(a,b,c1,c2,n)-padding2
print (m)

6. coppersmith short-pad attack

7. 私钥d 较小时(d<N^0.292)：Boneh and Durfee attack